In [3]:
import pandas as pd
import sqlite3

In [4]:
con = sqlite3.connect('proj6_readings.sqlite')
cur = con.cursor()

result = cur.execute("select count(*) from readings;").fetchall()

df = pd.DataFrame(result)

df

,0
0,50000000


In [5]:
df = pd.read_sql('select count(*) from readings;', con)
df

,count(*)
0,50000000


## Exercise 1: Basic counting

In [9]:
result1 = cur.execute('select count(distinct detector_id) from readings').fetchall()
ex01_df = pd.DataFrame(result1)
ex01_df

In [10]:
ex01_df.to_pickle('proj6_ex01_detector_no.pkl')

## Exercise 2: Some stats for the detectors

In [18]:
result2 = cur.execute('select detector_id, count(count), min(starttime), max(starttime) from readings group by(detector_id)'
                      ).fetchall()
ex02_df = pd.DataFrame(result2, columns=['detector_id', 'count(count)', 'min(starttime)', 'max(starttime']).reset_index(drop=True)
ex02_df

,detector_id,count(count),min(starttime),max(starttime
141,142,352730,2015-06-22 23:58:31,2016-07-12 23:57:01
142,143,352729,2015-06-22 23:58:31,2016-07-12 23:57:01
143,144,352730,2015-06-22 23:58:31,2016-07-12 23:57:01
144,145,352730,2015-06-22 23:58:31,2016-07-12 23:57:01
145,146,147666,2015-06-22 23:58:31,2015-12-01 04:21:01


In [20]:
ex02_df.to_pickle('proj6_ex02_detector_stat.pkl')

## Exercise 3: Moving Window

In [21]:
query3 = '''
SELECT detector_id, count, LAG(count) OVER (PARTITION BY detector_id ORDER BY starttime) AS prev_count
FROM readings
WHERE detector_id = 146
LIMIT 500;
'''

In [22]:
result3 = cur.execute(query3).fetchall()
ex03_df = pd.DataFrame(result3, columns=['detector_id', 'count', 'prev_count'])
ex03_df

,detector_id,count,prev_count
0,146,1,NaN
1,146,0,1.0
2,146,0,0.0
3,146,0,0.0
4,146,0,0.0
...,...,...,...
495,146,0,0.0
496,146,2,0.0
497,146,2,2.0
498,146,2,2.0


In [23]:
ex03_df.to_pickle('proj6_ex03_detector_146_lag.pkl')

## Exercise 4: Window

In [39]:
query4 = '''
select detector_id, count, sum(count) over (partition by detector_id order by starttime rows between current row and 10 following) as window_sum
from readings
where detector_id = 146
limit 500;
'''

In [40]:
result4 = cur.execute(query4).fetchall()
ex04_df = pd.DataFrame(result4, columns=['detector_id', 'count', 'window_sum'])
ex04_df

,detector_id,count,window_sum
0,146,1,2
1,146,0,2
2,146,0,2
3,146,0,2
4,146,0,2
...,...,...,...
495,146,0,20
496,146,2,22
497,146,2,21
498,146,2,21


In [41]:
ex04_df.to_pickle('proj6_ex04_detector_146_sum.pkl')